In [56]:
import psycopg2
import pandas as pd
import numpy as np
import time

In [17]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [9]:
try:
    connection = psycopg2.connect(database = "test_bismarck",
                                 user = "bismarckvm",
#                                  password = ConfigServer['pg_pass'],
                                 host= "127.0.0.1",
                                 port = "5432")
    cursor =connection.cursor()
    select_query = "select * from forest limit 10;"
    cursor.execute(select_query)
    print("Selecting rows from forest table using cursor.fetchall")
    forest_records = cursor.fetchall() 
    
    df = pd.read_sql(query, sql_conn)
    
    print("Print each row and it's columns values")
    for row in forest_records:
        print(row)
    print(type(row))

    
except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Selecting rows from forest table using cursor.fetchall
Print each row and it's columns values
(1, [-1.44068432237, 1.45060492878, -0.013848380928, -1.12646025152, -0.659040538501, -0.814590347924, -1.05140474936, 0.186203442816, 1.05741420528, 2.3143860444, 1.10807854775, -0.232859173676, -0.879364061489, -0.260673038712, -0.0724161768829, -0.114548140024, -0.09149045917, -0.147649464702, -0.0524997373724, -0.106986528008, -0.0134443535188, -0.0175549870083, -0.0444751337651, -0.243946665508, -0.147734456595, -0.233215935909, -0.175866697931, -0.0321251402219, -0.0022723131967, -0.0701478703348, -0.0769713911574, -0.057263897065, -0.0834799517022, -0.127255885265, -0.0380051330157, -0.246859004388, -0.332219240166, -0.194972661886, -0.0285741605343, -0.0669025863079, -0.0432740725992, -0.0403835994256, -0.49742954444, 4.27292984014, -0.214979341357, -0.315238035453, -0.290283313101, -0.0527300181234, -0.0571427818979, -0.0143128183985, -0.0226530044388, -0.165955865963, -0.156013840467

In [269]:
start = time.time()

connection = psycopg2.connect(database = "test_bismarck",
                             user = "bismarckvm",
                             host= "127.0.0.1",
                             port = "5432")
select_query = "select * from forest;"
forest_df = pd.read_sql(select_query, connection)


# X = forest_df["vec"].values
X = list(forest_df["vec"])
X = np.array(X)
y = forest_df["labeli"].values




clf = LogisticRegression(random_state=0, solver='liblinear',l1_ratio=1e-2, tol=1e-16).fit(X, y)


w = (clf.coef_).reshape(-1, len(clf.coef_))
loss = np.log(1 + np.exp(np.multiply((np.dot(X, w) + clf.intercept_),-y.reshape(len(y),-1))))
# np.log((1 + np.exp(-((np.dot(clf.coef_, X[0]) + clf.intercept_) * y[0]))))
sum(loss)

print("--- %s seconds ---" % (time.time() - start))

/home/mengyun/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:1506: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))


--- 171.1486053466797 seconds ---


In [270]:
sum(loss)

array([298341.52838183])

In [271]:
clf.score(X, y)

0.7561358457312414

In [136]:
# forest_df['vec'] = forest_df['vec'].apply(lambda x: np.asarray(x))
# forest_df['vec'] = forest_df['vec'].apply(lambda x: np.reshape(x, (-1, x.shape[0])))

In [288]:

class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h))
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            print("At iter: ", i)
            for j in range(len(X)):
                x = X[j]
                z = np.dot(x, self.theta)

                h = self.__sigmoid(z)

                gradient = np.dot(x.T, (h - y[j]))

                self.theta -= self.lr * gradient

                if(self.verbose == True and i % 10000 == 0):
                    z = np.dot(x, self.theta)
                    h = self.__sigmoid(z)
                    print("Loss: ", self.__loss(h,y))
                    
#             print("Loss: ", self.__loss(h,y))
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold=0.5):
        return self.predict_prob(X) >= threshold

In [289]:
start = time.time()
model = LogisticRegression(lr=0.1, num_iter=20)  
model.fit(X, y)

print("--- %s seconds ---" % (time.time() - start))

At iter:  0
At iter:  1
At iter:  2
At iter:  3
At iter:  4
At iter:  5
At iter:  6
At iter:  7
At iter:  8
At iter:  9
At iter:  10
At iter:  11
At iter:  12
At iter:  13
At iter:  14
At iter:  15
At iter:  16
At iter:  17
At iter:  18
At iter:  19
--- 251.86678194999695 seconds ---
